In [90]:
from random import random
import math
import csv


## Data Initialsation

In [91]:
# inputs (distance)
x = []
y = []

# outputs (velocity)
vx = []
vy = []

# parse the data into the set containers
with open('ce889_dataCollectionN.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=",")
    for line in csv_reader:
        x.append(float(line[0]))
        y.append(float(line[1]))
        vx.append(float(line[2]))
        vy.append(float(line[3]))

inputs = list(zip(x,y))
outputs = list(zip(vx,vy))

## Neuron Creation

In [92]:
class neuron:
        
    def sigmoid(self,inputs):
        sig = 1 / (1+math.exp(-inputs))
        return sig
    
    def activate(self, prev_layer, index):
        total = 0
        for i in range(0, len(prev_layer)):
            total = total + float(prev_layer[i].activationValue) * prev_layer[i].weights[index]
            self.activationValue = self.sigmoid(total)
    def derivate(self):
            return self.activationValue * (1.0 - self.activationValue)
    def __init__ (self, activationValue, numberOfweights, layerIndex):
        self.activationValue = activationValue
        self.weights= [random() for i in range(numberOfweights + 1)]
        self.delta_weight = [0,0]
        self.grad_val = 0
        self.layerIndex = layerIndex

In [93]:
# Debugging ( just a cat passing by 🐈)
randCat = [random() for i in range(5+1)]
print (randCat)

[0.3100946598666193, 0.16722833944650373, 0.5836033756774939, 0.529706238508763, 0.4760524861497334, 0.22459780081094205]


## Network Initialsation

In [94]:
# initialsation of the network
bias = -0.5
n_hiddenNeurons = 3
n_outNeurons = 2

# Network
# network = [neuron(0,n_hiddenNeurons,0),
#            neuron(0,n_hiddenNeurons,0),
#            neuron(0,n_outNeurons,1),
#            neuron(0,n_outNeurons,1),
#            neuron(0,n_outNeurons,1),
#            neuron(0,0,2),
#            neuron(0,0,2)
#           ]

input_layer = [neuron(0,n_hiddenNeurons,0),
               neuron(0,n_hiddenNeurons,0)]

hidden_layer = [neuron(0,n_outNeurons,1),
                neuron(0,n_outNeurons,1),
                neuron(0,n_outNeurons,1)]

output_layer = [neuron(0,0,2),
                neuron(0,0,2)]



In [95]:
# manual weight adjustment

# input_layer[0].weights = [0.1,0.1,0.1,bias]
# input_layer[1].weights = [0.1,0.1,0.1,bias]

# Add Bias to the end of the weight list
# hidden_layer[0].weights = [0.7,0.7,bias]
# hidden_layer[1].weights = [0.5,0.5,bias]
# hidden_layer[2].weights = [0.6,0.6,bias]

## Forward Propagation

In [96]:
def feedForward(inputs):
    
    outputActivationValues = []
    
    # injecting inputs into input layer 💉
    input_layer[0].activationValue = inputs[0]
    input_layer[1].activationValue = inputs[1]
        
    # activating the hidden layer
    for i in range(len(hidden_layer)):
        hidden_layer[i].activate(input_layer,(i-1))
        
    # feed forward
    output_layer[0].activate(hidden_layer,0)
    output_layer[1].activate(hidden_layer,1)
    
    # return activation values of output neurons
    outputActivationValues = [output_layer[0].activationValue, output_layer[1].activationValue]
    return outputActivationValues


for i in range(len(inputs)):
    print(i, feedForward(inputs[i]))

# Bad scaling!

0 [0.5669983213981199, 0.7601161014547736]
1 [0.5669983213981199, 0.7601161014547736]
2 [0.5669946476372418, 0.7601052459131484]
3 [0.5669969643766702, 0.7601118685472541]
4 [0.5669956073658786, 0.7601076356222499]
5 [0.5670002408053811, 0.7601208805885682]
6 [0.5670012005075525, 0.7601232701310594]
7 [0.5670081504615088, 0.7601431360362502]
8 [0.5670114267219503, 0.7601521468886557]
9 [0.5670206928574508, 0.7601786312454131]
10 [0.5670262853559412, 0.7601942611031872]
11 [0.5670378672068799, 0.7602273603712595]
12 [0.5670457755281044, 0.7602496059887137]
13 [0.567059672471119, 0.7602893155071994]
14 [0.5670698960038907, 0.7603181724049755]
15 [0.5670861072760566, 0.7603644864443099]
16 [0.5670986452692542, 0.7603999490735247]
17 [0.5671171699423099, 0.7604528607635954]
18 [0.567132021522658, 0.7604949225612008]
19 [0.5671528585136507, 0.7605544239238767]
20 [0.567170022603696, 0.7606030770749251]
21 [0.5671940117231438, 0.7606716214156339]
22 [0.5672160060779339, 0.7607342307303939]
2

## Backward Propagation

In [97]:
def BackPropagation(outputs):
    
    error = []
    lamba = 0.5
    lr = 0.6
    momentum = 0.7
    deltaWeights = []
    
    # error = (output - expected) * derivate
    for i in range(len(output_layer)):
        e = output_layer[i].activationValue - outputs[i]
        error.append(e)
    
    # calculating the slope of the output layer(2 iterations)
    for i in range(len(output_layer)):
        output_layer[i].grad_val = lamba * output_layer[i].derivate() * error[i]
        
    # calculating the slope of the hidden layer    
    for i in range(len(hidden_layer)):
        g = lamba * hidden_layer[i].derivate()
        result = 0
        for j in range(len(output_layer)):
            result += (output_layer[j].grad_val * hidden_layer[i].weights[j])
        hidden_layer[i].grad_val = g * result
        
    # calculates hidden layer delta weights    
    for z in range(len(hidden_layer)):
        for y in range(len(output_layer)):
            #error backprop: dw(delta weight) -->  back propped error 
            hidden_layer[z].delta_weight[y] = lr * output_layer[y].grad_val * hidden_layer[y].activationValue + momentum * hidden_layer[z].delta_weight[y]
            deltaWeights.append(hidden_layer[z].delta_weight[y])
            
    # calculates input layer delta weights 
    for y in range(len(input_layer)):
        for a in range(len(hidden_layer)-1):
            input_layer[y].delta_weight[a] = lr * hidden_layer[a].grad_val * input_layer[a].activationValue + momentum * input_layer[y].delta_weight[a]
            
    # updates the hidden layer weights
    for x in range(len(hidden_layer)):
        for i in range(len(output_layer)-1):
            hidden_layer[y].weights[i] = hidden_layer[x].delta_weight[i] * hidden_layer[x].weights[i]
            
    # updates the input layer weights        
    for y in range(len(input_layer)):
        for a in range(len(hidden_layer)-1):
            input_layer[y].weights[a] = input_layer[y].delta_weight[a] * input_layer[y].weights[a]
            
    # returns the hidden layer delta weights        
    return deltaWeights
            
for i in range(0, len(outputs)):
    print(i, BackPropagation(outputs[i]))


0 [0.011395986842503068, -0.006385760266887021, 0.011395986842503068, -0.006385760266887021, 0.011395986842503068, -0.006385760266887021]
1 [0.01975523046632897, -0.011550812954267594, 0.01975523046632897, -0.011550812954267594, 0.01975523046632897, -0.011550812954267594]
2 [0.02598875383708086, -0.014471329334874338, 0.02598875383708086, -0.014471329334874338, 0.02598875383708086, -0.014471329334874338]
3 [0.03073427307577095, -0.017210711301858715, 0.03073427307577095, -0.017210711301858715, 0.03073427307577095, -0.017210711301858715]
4 [0.034438189376927775, -0.01843325817818812, 0.034438189376927775, -0.01843325817818812, 0.034438189376927775, -0.01843325817818812]
5 [0.03741298362181131, -0.019984061492178365, 0.03741298362181131, -0.019984061492178365, 0.03741298362181131, -0.019984061492178365]
6 [0.03987739242730354, -0.020374603311411877, 0.03987739242730354, -0.020374603311411877, 0.03987739242730354, -0.020374603311411877]
7 [0.041984531425221856, -0.02134300308543499, 0.041